# Author imformation:
Fan Shengzhe, Shanghaijiaotong University, Shanghai, China  
Email: fanshengzhe@sjtu.edu.cn

# 1.读取数据

## 1.1 基本数据读取

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("drive/My Drive/reco/veg-fru-reco")

In [ ]:
import pandas as pd
from functools import partial
import os
import numpy as np
import gc


log_table = pd.read_csv('./cache/log_table.csv')
save_dir = './cache'

if not os.path.exists(save_dir):
  os.mkdir(save_dir)
item_info_raw_table = pd.read_csv('./cache/item_info_raw_table.csv')
log_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1630633 entries, 0 to 1630632
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   user_id        1630633 non-null  int64 
 1   item_id        1630633 non-null  int64 
 2   timestamp_ms   1630633 non-null  int64 
 3   environment    1630633 non-null  int64 
 4   device_group   1630633 non-null  int64 
 5   os             1630633 non-null  int64 
 6   province       1630633 non-null  int64 
 7   city           1630633 non-null  int64 
 8   referrer_type  1630633 non-null  int64 
 9   timestamp_s    1630633 non-null  object
dtypes: int64(9), object(1)
memory usage: 124.4+ MB


In [ ]:
log_table

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s
0,0,160417,1507029570190,0,0,0,0,0,0,2017-10-03 11:19:30
1,0,5408,1507029571478,0,0,0,0,0,0,2017-10-03 11:19:31
2,0,50823,1507029601478,0,0,0,0,0,0,2017-10-03 11:20:01
3,1,157770,1507029532200,0,0,0,0,1,1,2017-10-03 11:18:52
4,1,96613,1507029671831,0,0,0,0,1,1,2017-10-03 11:21:11
...,...,...,...,...,...,...,...,...,...,...
1630628,228075,70758,1508211323220,0,1,1,0,1,2,2017-10-17 03:35:23
1630629,242176,331116,1508211542618,0,1,1,0,1,0,2017-10-17 03:39:02
1630630,242176,234481,1508211850103,0,1,1,0,1,0,2017-10-17 03:44:10
1630631,242176,211442,1508212189949,0,1,1,0,1,0,2017-10-17 03:49:49


In [ ]:
item_info_raw_table

,item_id,category_id,prod_at_ts_ms,price,prod_at_ts_s
0,0,0,1513144419000,0.025112,2017-12-13 05:53:39
1,1,1,1405341936000,0.028251,2014-07-14 12:45:36
2,2,1,1408667706000,0.037369,2014-08-22 00:35:06
3,3,1,1408468313000,0.034380,2014-08-19 17:11:53
4,4,1,1407071171000,0.024215,2014-08-03 13:06:11
...,...,...,...,...,...
364042,364042,460,1434034118000,0.021525,2015-06-11 14:48:38
364043,364043,460,1434148472000,0.069208,2015-06-12 22:34:32
364044,364044,460,1457974279000,0.026457,2016-03-14 16:51:19
364045,364045,460,1515964737000,0.018834,2018-01-14 21:18:57


In [ ]:
log_table['timestamp_s'].max(), log_table['timestamp_s'].min()

('2017-11-13 20:04:14', '2017-10-02 15:44:10')

In [ ]:
item_info_raw_table['prod_at_ts_s'].max(), item_info_raw_table['prod_at_ts_s'].min()

('2018-03-13 12:12:30', '2006-09-27 11:14:35')

In [ ]:
try:
  from pandarallel import pandarallel
except:
  !pip install pandarallel
  from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16678 sha256=e2d13478e3be1b3c0895fe79c6d1e20d8a4389e48710321dbcd637a5c016f437
  Stored in directory: /root/.cache/pip/wheels/62/1e/e7/f9ee096e5cc02890a6934a5670ff6e45a3400f330605bd8210
Successfully built pandarallel
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# 节省内存的一个函数
# 减少内存
def reduce_mem(df):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
    col_type = df[col].dtypes
    if col_type in numerics:
      c_min = df[col].min()
      c_max = df[col].max()
      if pd.isnull(c_min) or pd.isnull(c_max):
        continue
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100*(start_mem-end_mem)/start_mem))
  return df

In [ ]:
log_table = reduce_mem(log_table)
item_info_raw_table = reduce_mem(item_info_raw_table)
gc.collect()

-- Mem. usage decreased to 46.65 Mb (62.5% reduction)
-- Mem. usage decreased to  8.33 Mb (40.0% reduction)


0

# 2.时间类特征

## 2.1 基础时间特征：年、月、日、时、分、秒

In [ ]:
import time
def get_year(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_year

def get_month(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_mon

def get_day(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_mday

def get_hour(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_hour

def get_minute(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_min

def get_second(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_sec

In [ ]:
# log_table['year'] = log_table['timestamp_s'].map(get_year)
# log_table['month'] = log_table['timestamp_s'].map(get_month)
log_table['day'] = log_table['timestamp_s'].map(get_day)
log_table['hour'] = log_table['timestamp_s'].map(get_hour)
log_table['minute'] = log_table['timestamp_s'].map(get_minute)
log_table['second'] = log_table['timestamp_s'].map(get_second)


item_info_raw_table['prod_year'] = item_info_raw_table['prod_at_ts_s'].map(get_year)
item_info_raw_table['prod_month'] = item_info_raw_table['prod_at_ts_s'].map(get_month)
item_info_raw_table['prod_day'] = item_info_raw_table['prod_at_ts_s'].map(get_day)
item_info_raw_table['prod_hour'] = item_info_raw_table['prod_at_ts_s'].map(get_hour)
item_info_raw_table['prod_minute'] = item_info_raw_table['prod_at_ts_s'].map(get_minute)
item_info_raw_table['prod_second'] = item_info_raw_table['prod_at_ts_s'].map(get_second)

## 2.2 季节类特征

In [ ]:
def timestamp2season(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  if (3 <= timeArray.tm_mon < 6):
    return 0  # spring
  elif (6 <= timeArray.tm_mon < 9):
    return 1  # summer 
  elif (9 <= timeArray.tm_mon < 12):
    return 2  # autumn
  elif (1 <= timeArray.tm_mon < 3 or timeArray.tm_mon == 12):
    return 3  # winter

In [ ]:
# log_table['season'] = log_table['timestamp_s'].map(timestamp2season)
item_info_raw_table['prod_season'] = item_info_raw_table['prod_at_ts_s'].map(timestamp2season)

## 2.3 时段类特征

In [ ]:
def timestamp2time_bucket(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  if (1 <= timeArray.tm_hour < 5):
    return 0  # 凌晨
  elif (5 <= timeArray.tm_hour < 8):
    return 1  # 早上
  elif (8 <= timeArray.tm_hour < 11):
    return 2  # 上午
  elif (11 <= timeArray.tm_hour < 13):
    return 3  # 中午
  elif (13 <= timeArray.tm_hour < 17):
    return 4  # 下午
  elif (17 <= timeArray.tm_hour < 19):
    return 5  # 傍晚
  elif (19 <= timeArray.tm_hour < 23):
    return 6  # 晚上
  elif (23 <= timeArray.tm_hour < 24 or 0 <= timeArray.tm_hour < 1):
    return 7  # 子夜

In [ ]:
log_table['time_bucket'] = log_table['timestamp_s'].map(timestamp2time_bucket)
item_info_raw_table['prod_time_bucket'] = item_info_raw_table['prod_at_ts_s'].map(timestamp2time_bucket)

## 2.4 星期类特征

In [ ]:
def timestamp2weekday(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_wday  # 0是周一

In [ ]:
log_table['weekday'] = log_table['timestamp_s'].map(timestamp2weekday)
item_info_raw_table['prod_weekday'] = item_info_raw_table['prod_at_ts_s'].map(timestamp2weekday)

## 2.5 年度特征（一年中的第几天）

In [ ]:
def timestamp2yearday(timestamp):
  timeArray = time.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return timeArray.tm_yday  # 0是周一

In [ ]:
log_table['yearday'] = log_table['timestamp_s'].map(timestamp2yearday)
item_info_raw_table['prod_yearday'] = item_info_raw_table['prod_at_ts_s'].map(timestamp2yearday)

## 2.6 merge item info

In [ ]:
log_table = pd.merge(log_table, item_info_raw_table, how='left', on='item_id')

In [ ]:
log_table

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s,...,prod_year,prod_month,prod_day,prod_hour,prod_minute,prod_second,prod_season,prod_time_bucket,prod_weekday,prod_yearday
0,0,160417,1507029570190,0,0,0,0,0,0,2017-10-03 11:19:30,...,2017,10,2,11,1,29,2,3,0,275
1,0,5408,1507029571478,0,0,0,0,0,0,2017-10-03 11:19:31,...,2017,10,3,1,30,57,2,0,1,276
2,0,50823,1507029601478,0,0,0,0,0,0,2017-10-03 11:20:01,...,2017,10,3,6,53,34,2,1,1,276
3,1,157770,1507029532200,0,0,0,0,1,1,2017-10-03 11:18:52,...,2017,10,2,22,38,55,2,6,0,275
4,1,96613,1507029671831,0,0,0,0,1,1,2017-10-03 11:21:11,...,2017,10,2,10,0,44,2,2,0,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630628,228075,70758,1508211323220,0,1,1,0,1,2,2017-10-17 03:35:23,...,2017,10,16,21,9,24,2,6,0,289
1630629,242176,331116,1508211542618,0,1,1,0,1,0,2017-10-17 03:39:02,...,2017,10,15,14,26,59,2,4,6,288
1630630,242176,234481,1508211850103,0,1,1,0,1,0,2017-10-17 03:44:10,...,2017,10,16,19,19,32,2,6,0,289
1630631,242176,211442,1508212189949,0,1,1,0,1,0,2017-10-17 03:49:49,...,2017,10,16,21,2,0,2,6,0,289


In [ ]:
log_table = reduce_mem(log_table)

-- Mem. usage decreased to 121.30 Mb (59.8% reduction)


## 2.7 得到freshness特征

In [ ]:
log_table['freshness'] = log_table['timestamp_ms'] - log_table['prod_at_ts_ms']

In [ ]:
log_table['freshness'].describe()

count    1.630633e+06
mean     8.353593e+08
std      7.629811e+09
min     -1.938483e+09
25%      1.912986e+07
50%      2.898531e+07
75%      5.694722e+07
max      3.404834e+11
Name: freshness, dtype: float64

In [ ]:
log_table = reduce_mem(log_table)

-- Mem. usage decreased to 133.74 Mb (0.0% reduction)


## 2.8 拆分历史数据

将用户的点击log拆分为：
* 历史行为序列
* 最后一次交互的物品

In [ ]:
def get_hist_and_last_click(all_click):
  all_click = all_click.sort_values(by=['user_id', 'timestamp_ms'])
  click_last_df = all_click.groupby('user_id').tail(1)

  def hist_func(group):
    last_purchase = group.tail(1)
    hist_click = pd.concat([group, last_purchase, last_purchase]).drop_duplicates(['user_id', 'item_id', 'timestamp_ms'], keep=False)
    
    if len(hist_click) == 0:
      return group
    else:
      return hist_click

  click_hist_df = all_click.groupby('user_id').parallel_apply(hist_func).reset_index(drop=True)

  return click_hist_df, click_last_df

In [ ]:
# 提取最后一次点击作为召回评估，如果不需要做召回评估直接使用全量的训练集进行召回(线下验证模型)
# 如果不是召回评估，直接使用全量数据进行召回，不用将最后一次提取出来
hist_click_df, last_click_df = get_hist_and_last_click(log_table)

hist_click_df = reduce_mem(hist_click_df)
last_click_df = reduce_mem(last_click_df)

-- Mem. usage decreased to 103.41 Mb (0.0% reduction)
-- Mem. usage decreased to 20.50 Mb (0.0% reduction)


# 3.用户侧统计特征

In [ ]:
hist_click_df['user_id'].nunique()

250000

In [ ]:
user_info = hist_click_df['user_id'].drop_duplicates().to_frame()
# user_info.info()

## 3.1 用户的时间偏好

### 3.1.1 event时间均值

In [ ]:
## 定义工具函数
def agg_mean(group, key):
  return pd.Series(group[key].mean(), index=[key])

def get_user_time_fea_by_mean(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除user_id和fea_name之外用到的特征列
  """
  in_cols += ['user_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mean, key=fea_name)
  data = data.groupby('user_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'u_mean_'+fea_name,}, inplace=True)
  
  return data[['user_id', 'u_mean_'+fea_name]]

In [ ]:
## 开始抽取特征
# 1. 基础时间特征
# # 1.1 年
# u_mean_year = get_user_time_fea_by_mean(hist_click_df, [], 'year')
# hist_click_df = pd.merge(hist_click_df, u_mean_year, how='outer', on='user_id')
# user_info = pd.merge(user_info, u_mean_year, how='outer', on='user_id')

# # 1.2 月
# u_mean_month = get_user_time_fea_by_mean(hist_click_df, [], 'month')
# hist_click_df = pd.merge(hist_click_df, u_mean_month, how='outer', on='user_id')
# user_info = pd.merge(user_info, u_mean_month, how='outer', on='user_id')

# 1.3 日
u_mean_day = get_user_time_fea_by_mean(hist_click_df, [], 'day')
hist_click_df = pd.merge(hist_click_df, u_mean_day, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_day, how='outer', on='user_id')

# 1.4 时
u_mean_hour = get_user_time_fea_by_mean(hist_click_df, [], 'hour')
hist_click_df = pd.merge(hist_click_df, u_mean_hour, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_hour, how='outer', on='user_id')

# 1.5 分
u_mean_minute = get_user_time_fea_by_mean(hist_click_df, [], 'minute')
hist_click_df = pd.merge(hist_click_df, u_mean_minute, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_minute, how='outer', on='user_id')

# 1.6 秒
u_mean_second = get_user_time_fea_by_mean(hist_click_df, [], 'second')
hist_click_df = pd.merge(hist_click_df, u_mean_second, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_second, how='outer', on='user_id')

# # 2. 季节类特征
# u_mean_season = get_user_time_fea_by_mean(hist_click_df, [], 'season')
# hist_click_df = pd.merge(hist_click_df, u_mean_season, how='outer', on='user_id')
# user_info = pd.merge(user_info, u_mean_season, how='outer', on='user_id')

# 3. 时段类特征
u_mean_time_bucket = get_user_time_fea_by_mean(hist_click_df, [], 'time_bucket')
hist_click_df = pd.merge(hist_click_df, u_mean_time_bucket, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_time_bucket, how='outer', on='user_id')

# 4. 星期类特征
u_mean_weekday = get_user_time_fea_by_mean(hist_click_df, [], 'weekday')
hist_click_df = pd.merge(hist_click_df, u_mean_weekday, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_weekday, how='outer', on='user_id')

# 5. 年度类特征
u_mean_yearday = get_user_time_fea_by_mean(hist_click_df, [], 'yearday')
hist_click_df = pd.merge(hist_click_df, u_mean_yearday, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_yearday, how='outer', on='user_id')

### 3.1.2 item创建时间均值

In [ ]:
## 开始抽取特征
# 1. 基础时间特征
# 1.1 年
u_mean_prod_year = get_user_time_fea_by_mean(hist_click_df, [], 'prod_year')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_year, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_year, how='outer', on='user_id')

# 1.2 月
u_mean_prod_month = get_user_time_fea_by_mean(hist_click_df, [], 'prod_month')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_month, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_month, how='outer', on='user_id')

# 1.3 日
u_mean_prod_day = get_user_time_fea_by_mean(hist_click_df, [], 'prod_day')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_day, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_day, how='outer', on='user_id')

# 1.4 时
u_mean_prod_hour = get_user_time_fea_by_mean(hist_click_df, [], 'prod_hour')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_hour, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_hour, how='outer', on='user_id')

# 1.5 分
u_mean_prod_minute = get_user_time_fea_by_mean(hist_click_df, [], 'prod_minute')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_minute, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_minute, how='outer', on='user_id')

# # 1.6 秒
# u_mean_prod_second = get_user_time_fea_by_mean(hist_click_df, [], 'prod_second')
# hist_click_df = pd.merge(hist_click_df, u_mean_prod_second, how='outer', on='user_id')
# user_info = pd.merge(user_info, u_mean_prod_second, how='outer', on='user_id')

# 2. 季节类特征
u_mean_prod_season = get_user_time_fea_by_mean(hist_click_df, [], 'prod_season')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_season, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_season, how='outer', on='user_id')

# 3. 时段类特征
u_mean_prod_time_bucket = get_user_time_fea_by_mean(hist_click_df, [], 'prod_time_bucket')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_time_bucket, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_time_bucket, how='outer', on='user_id')

# 4. 星期类特征
u_mean_prod_weekday = get_user_time_fea_by_mean(hist_click_df, [], 'prod_weekday')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_weekday, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_weekday, how='outer', on='user_id')

# 5. 年度类特征
u_mean_prod_yearday = get_user_time_fea_by_mean(hist_click_df, [], 'prod_yearday')
hist_click_df = pd.merge(hist_click_df, u_mean_prod_yearday, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_prod_yearday, how='outer', on='user_id')

In [ ]:
hist_click_df.describe()

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,day,...,u_mean_yearday,u_mean_prod_year,u_mean_prod_month,u_mean_prod_day,u_mean_prod_hour,u_mean_prod_minute,u_mean_prod_season,u_mean_prod_time_bucket,u_mean_prod_weekday,u_mean_prod_yearday
count,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,...,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06
mean,1.243686e+05,1.944529e+05,1.507492e+12,4.838388e-02,4.020313e-01,5.838012e-01,1.558159e-01,6.631805e+00,1.251020e+00,8.215457e+00,...,2.812157e+02,2.016983e+03,9.929924e+00,8.055305e+00,1.238473e+01,2.722265e+01,1.988032e+00,3.254712e+00,2.236585e+00,2.789279e+02
std,8.367476e+04,9.068042e+04,3.515273e+08,3.052072e-01,5.580647e-01,8.863815e-01,8.833514e-01,6.649000e+00,1.069184e+00,4.092337e+00,...,2.775081e+00,1.318043e-01,3.996181e-01,2.916622e+00,2.642829e+00,7.848415e+00,9.785104e-02,9.108809e-01,1.132119e+00,1.185457e+01
min,0.000000e+00,6.900000e+01,1.506959e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,2.750000e+02,2.007000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,4.399400e+04,1.241940e+05,1.507166e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,...,2.794091e+02,2.017000e+03,1.000000e+01,6.219512e+00,1.085106e+01,2.350000e+01,2.000000e+00,2.733333e+00,1.510638e+00,2.782632e+02
50%,1.160760e+05,2.023550e+05,1.507516e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,2.000000e+00,9.000000e+00,...,2.811667e+02,2.017000e+03,1.000000e+01,8.000000e+00,1.230000e+01,2.726087e+01,2.000000e+00,3.224490e+00,2.142857e+00,2.803684e+02
75%,2.105440e+05,2.769700e+05,1.507743e+12,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+01,2.000000e+00,1.100000e+01,...,2.830000e+02,2.017000e+03,1.000000e+01,9.714286e+00,1.386667e+01,3.083333e+01,2.000000e+00,3.769231e+00,2.888889e+00,2.822500e+02
max,2.499990e+05,3.640460e+05,1.510603e+12,2.000000e+00,4.000000e+00,7.000000e+00,1.000000e+01,2.700000e+01,6.000000e+00,3.100000e+01,...,3.022857e+02,2.017000e+03,1.200000e+01,3.100000e+01,2.300000e+01,5.900000e+01,3.000000e+00,7.000000e+00,6.000000e+00,3.660000e+02


## 3.2 用户的价格偏好

In [ ]:
u_mean_price = get_user_time_fea_by_mean(hist_click_df, [], 'price')
hist_click_df = pd.merge(hist_click_df, u_mean_price, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_price, how='outer', on='user_id')

## 3.3 用户的类别偏好

In [ ]:
import random
import numpy as np
def agg_mode(group, key):
  mode_list = group[key].mode()
  if not mode_list.empty:
    return pd.Series(random.choice(mode_list), index=[key])
  else:
    return pd.Series(np.nan, index=[key])

def get_user_time_fea_by_mode(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除user_id和fea_name之外用到的特征列
  """
  in_cols += ['user_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mode, key=fea_name)
  data = data.groupby('user_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'u_mode_'+fea_name,}, inplace=True)
  
  return data[['user_id', 'u_mode_'+fea_name]]

In [ ]:
u_mode_category_id = get_user_time_fea_by_mode(hist_click_df, [], 'category_id')
hist_click_df = pd.merge(hist_click_df, u_mode_category_id, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_category_id, how='outer', on='user_id')

## 3.4 用户的浏览环境偏好

In [ ]:
u_mode_environment = get_user_time_fea_by_mode(hist_click_df, [], 'environment')
hist_click_df = pd.merge(hist_click_df, u_mode_environment, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_environment, how='outer', on='user_id')

## 3.5 用户的设备偏好

In [ ]:
u_mode_device = get_user_time_fea_by_mode(hist_click_df, [], 'device_group')
hist_click_df = pd.merge(hist_click_df, u_mode_device, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_device, how='outer', on='user_id')

## 3.6 用户的系统偏好

In [ ]:
u_mode_os = get_user_time_fea_by_mode(hist_click_df, [], 'os')
hist_click_df = pd.merge(hist_click_df, u_mode_os, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_os, how='outer', on='user_id')

## 3.7 用户的省市偏好

In [ ]:
u_mode_province = get_user_time_fea_by_mode(hist_click_df, [], 'province')
hist_click_df = pd.merge(hist_click_df, u_mode_province, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_province, how='outer', on='user_id')

In [ ]:
u_mode_city = get_user_time_fea_by_mode(hist_click_df, [], 'city')
hist_click_df = pd.merge(hist_click_df, u_mode_city, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_city, how='outer', on='user_id')

## 3.8 用户的浏览场景偏好

In [ ]:
u_mode_referrer_type = get_user_time_fea_by_mode(hist_click_df, [], 'referrer_type')
hist_click_df = pd.merge(hist_click_df, u_mode_referrer_type, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mode_referrer_type, how='outer', on='user_id')

## 3.9 用户的活跃度特征

活跃度：
1. 购买行为的个数
2. 购买行为活跃天数
3. 平均购买时间间隔
4. 购买行为个数除以时间间隔

### 3.9.1 购买行为的活跃度

In [ ]:
import time
## 定义工具函数
def str2timestamp(time_str):
  timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
  timeStamp = int(time.mktime(timeArray))
  return timeStamp

def get_ua_nums(group):
  return pd.Series([len(group)], index=['ua_nums'])

def get_user_active_by_deal_nums(data, in_cols):
  in_cols += ['user_id']
  data = data[in_cols]
  agg_func = get_ua_nums

  data = data.groupby('user_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={'ua_nums': 'u_active_nums'}, inplace=True)
  
  return data[['user_id', 'u_active_nums']]

In [ ]:
u_active_nums = get_user_active_by_deal_nums(hist_click_df, ['timestamp_ms'])
hist_click_df = pd.merge(hist_click_df, u_active_nums, how='outer', on='user_id')
user_info = pd.merge(user_info, u_active_nums, how='outer', on='user_id')

### 3.9.2 购买行为的活跃天数

In [ ]:
import time
## 定义工具函数
def str2ymd(time_str):
  timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
  return time.strftime("%Y-%m-%d", timeArray)

def get_deal_day_num(group):
  return pd.Series(group['timestamp_s'].nunique(), index=['deal_day_num'])

def get_user_active_by_deal_nums(data, in_cols):
  in_cols += ['user_id']
  data = data[in_cols]
  agg_func = get_deal_day_num

  data['timestamp_s'] = data['timestamp_s'].parallel_apply(str2ymd)
  data = data.groupby('user_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={'deal_day_num': 'u_active_day_nums'}, inplace=True)
  
  return data[['user_id', 'u_active_day_nums']]

In [ ]:
u_active_deal_day_nums = get_user_active_by_deal_nums(hist_click_df, ['timestamp_s'])
hist_click_df = pd.merge(hist_click_df, u_active_deal_day_nums, how='outer', on='user_id')
user_info = pd.merge(user_info, u_active_deal_day_nums, how='outer', on='user_id')

<ipython-input-46-45f2f7220f7f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['timestamp_s'] = data['timestamp_s'].parallel_apply(str2ymd)


### 3.9.3 平均ua时间倒数

In [ ]:
def get_mean_ua_diff(group):
  if len(group) == 1:
    return pd.Series(np.nan, index=['ua_diff'])
  else:
    return pd.Series((group['timestamp_ms']-group['time_stamp_ms_diff']).mean(), index=['ua_diff'])


def rev(num):
  return 1 / num


def get_ua_time_diff_by_mean(data, in_cols):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除user_id和fea_name之外用到的特征列
  """
  in_cols += ['user_id']
  data = data[in_cols]
  agg_func = get_mean_ua_diff
  data['time_stamp_ms_diff'] = data['timestamp_ms'].diff()
  data = data.groupby('user_id').parallel_apply(agg_func).reset_index()

  data['ua_diff'] = data['ua_diff'].map(rev)
  min_ua_diff, max_ua_diff = data['ua_diff'].min(), data['ua_diff'].max()
  data['ua_diff'] = (data['ua_diff'] - min_ua_diff) / (max_ua_diff - min_ua_diff)
  data.rename(columns={'ua_diff': 'u_mean_rev_ua_time'}, inplace=True)
  
  return data[['user_id', 'u_mean_rev_ua_time']]

In [ ]:
u_mean_rev_ua_time = get_ua_time_diff_by_mean(hist_click_df, ['timestamp_ms']).fillna(0.)
hist_click_df = pd.merge(hist_click_df, u_mean_rev_ua_time, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_rev_ua_time, how='outer', on='user_id')

<ipython-input-48-3710cf472530>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time_stamp_ms_diff'] = data['timestamp_ms'].diff()


### 3.9.4 ua次数+ua时间倒数

In [ ]:
u_ua_score = pd.merge(u_active_nums, u_mean_rev_ua_time, how='inner', on='user_id')
u_ua_score['u_active_nums'] = (u_ua_score['u_active_nums'] - u_ua_score['u_active_nums'].min()) / \
              (u_ua_score['u_active_nums'].max() - u_ua_score['u_active_nums'].min())
u_ua_score['u_ua_score'] = 0.5 * u_ua_score['u_active_nums'] + 0.5 * u_ua_score['u_mean_rev_ua_time']
u_ua_score['u_ua_score'] = (u_ua_score['u_ua_score'] - u_ua_score['u_ua_score'].min()) / \
              (u_ua_score['u_ua_score'].max() - u_ua_score['u_ua_score'].min())

u_ua_score = u_ua_score[['user_id', 'u_ua_score']]
hist_click_df = pd.merge(hist_click_df, u_ua_score, how='outer', on='user_id')
user_info = pd.merge(user_info, u_ua_score, how='outer', on='user_id')

## 3.10 用户的freshness偏好

In [ ]:
u_mean_freshness = get_user_time_fea_by_mean(hist_click_df, [], 'freshness')
hist_click_df = pd.merge(hist_click_df, u_mean_freshness, how='outer', on='user_id')
user_info = pd.merge(user_info, u_mean_freshness, how='outer', on='user_id')

In [ ]:
hist_click_df = reduce_mem(hist_click_df)
user_info = reduce_mem(user_info)

-- Mem. usage decreased to 184.28 Mb (47.3% reduction)
-- Mem. usage decreased to 15.50 Mb (65.8% reduction)


# 4.物品侧统计特征

In [ ]:
item_info_raw_table

,item_id,category_id,prod_at_ts_ms,price,prod_at_ts_s,prod_year,prod_month,prod_day,prod_hour,prod_minute,prod_second,prod_season,prod_time_bucket,prod_weekday,prod_yearday
0,0,0,1513144419000,0.025116,2017-12-13 05:53:39,2017,12,13,5,53,39,3,1,2,347
1,1,1,1405341936000,0.028244,2014-07-14 12:45:36,2014,7,14,12,45,36,1,3,0,195
2,2,1,1408667706000,0.037384,2014-08-22 00:35:06,2014,8,22,0,35,6,1,7,4,234
3,3,1,1408468313000,0.034393,2014-08-19 17:11:53,2014,8,19,17,11,53,1,5,1,231
4,4,1,1407071171000,0.024216,2014-08-03 13:06:11,2014,8,3,13,6,11,1,4,6,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364042,364042,460,1434034118000,0.021530,2015-06-11 14:48:38,2015,6,11,14,48,38,1,4,3,162
364043,364043,460,1434148472000,0.069214,2015-06-12 22:34:32,2015,6,12,22,34,32,1,6,4,163
364044,364044,460,1457974279000,0.026459,2016-03-14 16:51:19,2016,3,14,16,51,19,0,4,0,74
364045,364045,460,1515964737000,0.018829,2018-01-14 21:18:57,2018,1,14,21,18,57,3,6,6,14


In [ ]:
item_info = item_info_raw_table.drop_duplicates()

## 4.1 物品被购买的时间偏好

### 4.1.1 成交行为均值

In [ ]:
# hist_click_df.describe()

In [ ]:
def get_item_time_fea_by_mean(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除item_id和fea_name之外用到的特征列
  """
  in_cols += ['item_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mean, key=fea_name)
  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'i_mean_'+fea_name,}, inplace=True)
  
  return data[['item_id', 'i_mean_'+fea_name]]

In [ ]:
## 开始抽取特征
# 1. 基础时间特征
# # 1.1 年
# i_mean_year = get_item_time_fea_by_mean(hist_click_df, [], 'year')
# hist_click_df = pd.merge(hist_click_df, i_mean_year, how='outer', on='item_id')
# item_info = pd.merge(item_info, i_mean_year, how='outer', on='item_id')

# # 1.2 月
# i_mean_month = get_item_time_fea_by_mean(hist_click_df, [], 'month')
# hist_click_df = pd.merge(hist_click_df, i_mean_month, how='outer', on='item_id')
# item_info = pd.merge(item_info, i_mean_month, how='outer', on='item_id')

# 1.3 日
i_mean_day = get_item_time_fea_by_mean(hist_click_df, [], 'day')
hist_click_df = pd.merge(hist_click_df, i_mean_day, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_day, how='outer', on='item_id')

# 1.4 时
i_mean_hour = get_item_time_fea_by_mean(hist_click_df, [], 'hour')
hist_click_df = pd.merge(hist_click_df, i_mean_hour, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_hour, how='outer', on='item_id')

# 1.5 分
i_mean_minute = get_item_time_fea_by_mean(hist_click_df, [], 'minute')
hist_click_df = pd.merge(hist_click_df, i_mean_minute, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_minute, how='outer', on='item_id')

# # 1.6 秒
# i_mean_second = get_item_time_fea_by_mean(hist_click_df, [], 'second')
# hist_click_df = pd.merge(hist_click_df, i_mean_second, how='outer', on='item_id')
# item_info = pd.merge(item_info, i_mean_second, how='outer', on='item_id')

# # 2. 季节类特征
# i_mean_season = get_item_time_fea_by_mean(hist_click_df, [], 'season')
# hist_click_df = pd.merge(hist_click_df, i_mean_season, how='outer', on='item_id')
# item_info = pd.merge(item_info, i_mean_season, how='outer', on='item_id')

# 3. 时段类特征
i_mean_time_bucket = get_item_time_fea_by_mean(hist_click_df, [], 'time_bucket')
hist_click_df = pd.merge(hist_click_df, i_mean_time_bucket, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_time_bucket, how='outer', on='item_id')

# 4. 星期类特征
i_mean_weekday = get_item_time_fea_by_mean(hist_click_df, [], 'weekday')
hist_click_df = pd.merge(hist_click_df, i_mean_weekday, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_weekday, how='outer', on='item_id')

# 5. 年度类特征
i_mean_yearday = get_item_time_fea_by_mean(hist_click_df, [], 'yearday')
hist_click_df = pd.merge(hist_click_df, i_mean_yearday, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_yearday, how='outer', on='item_id')

## 4.2 物品被购买的浏览环境偏好

In [ ]:
def get_item_time_fea_by_mode(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除item_id和fea_name之外用到的特征列
  """
  in_cols += ['item_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mode, key=fea_name)
  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'i_mode_'+fea_name,}, inplace=True)
  
  return data[['item_id', 'i_mode_'+fea_name]]

In [ ]:
i_mode_environment = get_item_time_fea_by_mode(hist_click_df, [], 'environment')
hist_click_df = pd.merge(hist_click_df, i_mode_environment, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_environment, how='outer', on='item_id')

## 4.3 物品被购买的设备偏好

In [ ]:
i_mode_device = get_item_time_fea_by_mode(hist_click_df, [], 'device_group')
hist_click_df = pd.merge(hist_click_df, i_mode_device, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_device, how='outer', on='item_id')

## 4.4 物品被购买的系统偏好

In [ ]:
i_mode_os = get_item_time_fea_by_mode(hist_click_df, [], 'os')
hist_click_df = pd.merge(hist_click_df, i_mode_os, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_os, how='outer', on='item_id')

## 4.5 物品被购买的省市偏好

In [ ]:
i_mode_province = get_item_time_fea_by_mode(hist_click_df, [], 'province')
hist_click_df = pd.merge(hist_click_df, i_mode_province, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_province, how='outer', on='item_id')

In [ ]:
i_mode_city = get_item_time_fea_by_mode(hist_click_df, [], 'city')
hist_click_df = pd.merge(hist_click_df, i_mode_city, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_city, how='outer', on='item_id')

## 4.6 物品被购买的浏览场景偏好

In [ ]:
i_mode_referrer_type = get_item_time_fea_by_mode(hist_click_df, [], 'referrer_type')
hist_click_df = pd.merge(hist_click_df, i_mode_referrer_type, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mode_referrer_type, how='outer', on='item_id')

## 4.7 物品的热度特征

### 4.7.1 购买行为的活跃度

In [ ]:
def get_item_active_by_nums(data, in_cols):
  in_cols += ['item_id']
  data = data[in_cols]
  agg_func = get_ua_nums

  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={'ua_nums': 'i_active_nums'}, inplace=True)
  
  return data[['item_id', 'i_active_nums']]

In [ ]:
i_active_nums = get_item_active_by_nums(hist_click_df, ['timestamp_ms'])
hist_click_df = pd.merge(hist_click_df, i_active_nums, how='outer', on='item_id')
item_info = pd.merge(item_info, i_active_nums, how='outer', on='item_id')

### 4.7.2 购买行为的活跃天数

In [ ]:
def get_item_active_by_nums(data, in_cols):
  in_cols += ['item_id']
  data = data[in_cols]
  agg_func = get_deal_day_num

  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={'deal_day_num': 'i_active_day_nums'}, inplace=True)
  
  return data[['item_id', 'i_active_day_nums']]

In [ ]:
i_active_day_nums = get_item_active_by_nums(hist_click_df, ['timestamp_s'])
hist_click_df = pd.merge(hist_click_df, i_active_day_nums, how='outer', on='item_id')
item_info = pd.merge(item_info, i_active_day_nums, how='outer', on='item_id')

### 4.7.3 平均ua时间倒数

In [ ]:
def get_item_ua_time_diff_by_mean(data, in_cols):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除item_id和fea_name之外用到的特征列
  """
  in_cols += ['item_id']
  data = data[in_cols]
  agg_func = get_mean_ua_diff
  data['time_stamp_ms_diff'] = data['timestamp_ms'].diff()
  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data['ua_diff'] = data['ua_diff'].map(rev)
  min_ua_diff, max_ua_diff = data['ua_diff'].min(), data['ua_diff'].max()
  data['ua_diff'] = (data['ua_diff'] - min_ua_diff) / (max_ua_diff - min_ua_diff)
  data.rename(columns={'ua_diff': 'i_mean_rev_ua_time'}, inplace=True)
  
  return data[['item_id', 'i_mean_rev_ua_time']]

In [ ]:
i_mean_rev_ua_time = get_item_ua_time_diff_by_mean(hist_click_df, ['timestamp_ms']).fillna(0.)
hist_click_df = pd.merge(hist_click_df, i_mean_rev_ua_time, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_rev_ua_time, how='outer', on='item_id')

<ipython-input-69-f7599b691680>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time_stamp_ms_diff'] = data['timestamp_ms'].diff()


### 4.7.4 ua次数+ua时间倒数

In [ ]:
item_ua_score = pd.merge(i_active_nums, i_mean_rev_ua_time, how='inner', on='item_id')
item_ua_score['i_active_nums'] = (item_ua_score['i_active_nums'] - item_ua_score['i_active_nums'].min()) / \
              (item_ua_score['i_active_nums'].max() - item_ua_score['i_active_nums'].min())
item_ua_score['i_ua_score'] = 0.5 * item_ua_score['i_active_nums'] + 0.5 * item_ua_score['i_mean_rev_ua_time']
item_ua_score['i_ua_score'] = (item_ua_score['i_ua_score'] - item_ua_score['i_ua_score'].min()) / \
              (item_ua_score['i_ua_score'].max() - item_ua_score['i_ua_score'].min())

item_ua_score = item_ua_score[['item_id', 'i_ua_score']]
hist_click_df = pd.merge(hist_click_df, item_ua_score, how='outer', on='item_id')
item_info = pd.merge(item_info, item_ua_score, how='outer', on='item_id')

## 4.8 物品成交时freshness统计信息

In [ ]:
i_mean_freshness = get_item_time_fea_by_mean(hist_click_df, [], 'freshness')
hist_click_df = pd.merge(hist_click_df, i_mean_freshness, how='outer', on='item_id')
item_info = pd.merge(item_info, i_mean_freshness, how='outer', on='item_id')

In [ ]:
hist_click_df = reduce_mem(hist_click_df)
item_info = reduce_mem(item_info)

-- Mem. usage decreased to 224.05 Mb (27.5% reduction)
-- Mem. usage decreased to 27.77 Mb (67.7% reduction)


# 5.物品消费者侧特征

In [ ]:
consumer_info = hist_click_df['item_id'].drop_duplicates().to_frame()

## 5.1 物品消费者的时间偏好

### 5.1.1 event时间均值

In [ ]:
# hist_click_df.describe()

In [ ]:
def get_consumer_time_fea_by_mean(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除user_id和fea_name之外用到的特征列
  """
  in_cols += ['item_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mean, key=fea_name)
  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'c_mean_'+fea_name,}, inplace=True)
  
  return data[['item_id', 'c_mean_'+fea_name]]

In [ ]:
## 开始抽取特征
# 1. 基础时间特征
# # 1.1 年
# c_mean_year = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_year')
# hist_click_df = pd.merge(hist_click_df, c_mean_year, how='outer', on='item_id')
# consumer_info = pd.merge(consumer_info, c_mean_year, how='outer', on='item_id')

# # 1.2 月
# c_mean_month = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_month')
# hist_click_df = pd.merge(hist_click_df, c_mean_month, how='outer', on='item_id')
# consumer_info = pd.merge(consumer_info, c_mean_month, how='outer', on='item_id')

# 1.3 日
c_mean_day = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_day')
hist_click_df = pd.merge(hist_click_df, c_mean_day, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_day, how='outer', on='item_id')

# 1.4 时
c_mean_hour = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_hour')
hist_click_df = pd.merge(hist_click_df, c_mean_hour, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_hour, how='outer', on='item_id')

# 1.5 分
c_mean_minute = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_minute')
hist_click_df = pd.merge(hist_click_df, c_mean_minute, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_minute, how='outer', on='item_id')

# # 1.6 秒
# c_mean_second = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_second')
# hist_click_df = pd.merge(hist_click_df, c_mean_second, how='outer', on='item_id')
# consumer_info = pd.merge(consumer_info, c_mean_second, how='outer', on='item_id')

# # 2. 季节类特征
# c_mean_season = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_season')
# hist_click_df = pd.merge(hist_click_df, c_mean_season, how='outer', on='item_id')
# consumer_info = pd.merge(consumer_info, c_mean_season, how='outer', on='item_id')

# 3. 时段类特征
c_mean_time_bucket = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_time_bucket')
hist_click_df = pd.merge(hist_click_df, c_mean_time_bucket, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_time_bucket, how='outer', on='item_id')

# 4. 星期类特征
c_mean_weekday = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_weekday')
hist_click_df = pd.merge(hist_click_df, c_mean_weekday, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_weekday, how='outer', on='item_id')

# 5. 年度类特征
c_mean_yearday = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_yearday')
hist_click_df = pd.merge(hist_click_df, c_mean_yearday, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_yearday, how='outer', on='item_id')

### 5.1.2 item创建时间均值

In [ ]:
## 开始抽取特征
# 1. 基础时间特征
# 1.1 年
c_mean_prod_year = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_year')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_year, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_year, how='outer', on='item_id')

# 1.2 月
c_mean_prod_month = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_month')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_month, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_month, how='outer', on='item_id')

# 1.3 日
c_mean_prod_day = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_day')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_day, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_day, how='outer', on='item_id')

# 1.4 时
c_mean_prod_hour = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_hour')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_hour, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_hour, how='outer', on='item_id')

# 1.5 分
c_mean_prod_minute = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_minute')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_minute, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_minute, how='outer', on='item_id')

# # 1.6 秒
# c_mean_prod_second = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_second')
# hist_click_df = pd.merge(hist_click_df, c_mean_prod_second, how='outer', on='item_id')
# consumer_info = pd.merge(consumer_info, c_mean_prod_second, how='outer', on='item_id')

# 2. 季节类特征
c_mean_prod_season = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_season')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_season, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_season, how='outer', on='item_id')

# 3. 时段类特征
c_mean_prod_time_bucket = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_time_bucket')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_time_bucket, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_time_bucket, how='outer', on='item_id')

# 4. 星期类特征
c_mean_prod_weekday = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_weekday')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_weekday, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_weekday, how='outer', on='item_id')

# 5. 年度类特征
c_mean_prod_yearday = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_prod_yearday')
hist_click_df = pd.merge(hist_click_df, c_mean_prod_yearday, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_prod_yearday, how='outer', on='item_id')

## 5.2 物品消费者的价格偏好

In [ ]:
c_mean_price = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_price')
hist_click_df = pd.merge(hist_click_df, c_mean_price, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_price, how='outer', on='item_id')

## 5.3 物品消费者的类别偏好

In [ ]:
def get_consumer_time_fea_by_mode(data, in_cols, fea_name):
  """
  制作用户的时间习惯特征
  :param data: 数据集
  :param extra_cols: 除user_id和fea_name之外用到的特征列
  """
  in_cols += ['item_id', fea_name]
  data = data[in_cols]
  agg_func = partial(agg_mode, key=fea_name)
  data = data.groupby('item_id').parallel_apply(agg_func).reset_index()

  data.rename(columns={fea_name: 'c_mode_'+fea_name,}, inplace=True)
  
  return data[['item_id', 'c_mode_'+fea_name]]

In [ ]:
c_mode_category_id = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_category_id')
hist_click_df = pd.merge(hist_click_df, c_mode_category_id, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_category_id, how='outer', on='item_id')

## 3.4 物品消费者的浏览环境偏好

In [ ]:
c_mode_environment = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_environment')
hist_click_df = pd.merge(hist_click_df, c_mode_environment, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_environment, how='outer', on='item_id')

## 3.5 物品消费者的设备偏好

In [ ]:
c_mode_device_group = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_device_group')
hist_click_df = pd.merge(hist_click_df, c_mode_device_group, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_device_group, how='outer', on='item_id')

## 3.6 物品消费者的系统偏好

In [ ]:
c_mode_os = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_os')
hist_click_df = pd.merge(hist_click_df, c_mode_os, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_os, how='outer', on='item_id')

## 3.7 物品消费者的省市偏好

In [ ]:
c_mode_province = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_province')
hist_click_df = pd.merge(hist_click_df, c_mode_province, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_province, how='outer', on='item_id')

In [ ]:
c_mode_city = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_city')
hist_click_df = pd.merge(hist_click_df, c_mode_city, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_city, how='outer', on='item_id')

## 3.8 物品消费者浏览场景偏好

In [ ]:
c_mode_referrer_type = get_consumer_time_fea_by_mode(hist_click_df, [], 'u_mode_referrer_type')
hist_click_df = pd.merge(hist_click_df, c_mode_referrer_type, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mode_referrer_type, how='outer', on='item_id')

## 5.6 物品消费者的活跃度特征

### 5.6.1 购买行为的活跃度

In [ ]:
c_active_nums = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_active_nums')
hist_click_df = pd.merge(hist_click_df, c_active_nums, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_active_nums, how='outer', on='item_id')

### 5.6.2 购买行为的活跃天数

In [ ]:
c_active_day_nums = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_active_day_nums')
hist_click_df = pd.merge(hist_click_df, c_active_day_nums, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_active_day_nums, how='outer', on='item_id')

### 5.6.3 平均ua时间倒数

In [ ]:
c_mean_rev_ua_time = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_rev_ua_time')
hist_click_df = pd.merge(hist_click_df, c_mean_rev_ua_time, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_rev_ua_time, how='outer', on='item_id')

### 5.6.4 ua次数+ua时间倒数

In [ ]:
c_ua_score = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_ua_score')
hist_click_df = pd.merge(hist_click_df, c_ua_score, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_ua_score, how='outer', on='item_id')

## 5.7 物品消费者的freshness偏好



In [ ]:
c_mean_freshness = get_consumer_time_fea_by_mean(hist_click_df, [], 'u_mean_freshness')
hist_click_df = pd.merge(hist_click_df, c_mean_freshness, how='outer', on='item_id')
consumer_info = pd.merge(consumer_info, c_mean_freshness, how='outer', on='item_id')

In [ ]:
hist_click_df = reduce_mem(hist_click_df)
consumer_info = reduce_mem(consumer_info)

-- Mem. usage decreased to 372.54 Mb (-20.6% reduction)
-- Mem. usage decreased to  3.62 Mb (-63.2% reduction)


# 6.预览

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

## 6.1 样本预览

In [ ]:
hist_click_df

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s,day,hour,minute,second,time_bucket,weekday,yearday,category_id,prod_at_ts_ms,price,prod_at_ts_s,prod_year,prod_month,prod_day,prod_hour,prod_minute,prod_second,prod_season,prod_time_bucket,prod_weekday,prod_yearday,freshness,u_mean_day,u_mean_hour,u_mean_minute,u_mean_second,u_mean_time_bucket,u_mean_weekday,u_mean_yearday,u_mean_prod_year,u_mean_prod_month,u_mean_prod_day,u_mean_prod_hour,u_mean_prod_minute,u_mean_prod_season,u_mean_prod_time_bucket,u_mean_prod_weekday,u_mean_prod_yearday,u_mean_price,u_mode_category_id,u_mode_environment,u_mode_device_group,u_mode_os,u_mode_province,u_mode_city,u_mode_referrer_type,u_active_nums,u_active_day_nums,u_mean_rev_ua_time,u_ua_score,u_mean_freshness,i_mean_day,i_mean_hour,i_mean_minute,i_mean_time_bucket,i_mean_weekday,i_mean_yearday,i_mode_environment,i_mode_device_group,i_mode_os,i_mode_province,i_mode_city,i_mode_referrer_type,i_active_nums,i_active_day_nums,i_mean_rev_ua_time,i_ua_score,i_mean_freshness,c_mean_u_mean_day,c_mean_u_mean_hour,c_mean_u_mean_minute,c_mean_u_mean_time_bucket,c_mean_u_mean_weekday,c_mean_u_mean_yearday,c_mean_u_mean_prod_year,c_mean_u_mean_prod_month,c_mean_u_mean_prod_day,c_mean_u_mean_prod_hour,c_mean_u_mean_prod_minute,c_mean_u_mean_prod_season,c_mean_u_mean_prod_time_bucket,c_mean_u_mean_prod_weekday,c_mean_u_mean_prod_yearday,c_mean_u_mean_price,c_mode_u_mode_category_id,c_mode_u_mode_environment,c_mode_u_mode_device_group,c_mode_u_mode_os,c_mode_u_mode_province,c_mode_u_mode_city,c_mode_u_mode_referrer_type,c_mean_u_active_nums,c_mean_u_active_day_nums,c_mean_u_mean_rev_ua_time,c_mean_u_ua_score,c_mean_u_mean_freshness
0,0,160417,1507029570190,0,0,0,0,0,0,2017-10-03 11:19:30,3,11,19,30,3,1,276,281,1506942089000,0.025864,2017-10-02 11:01:29,2017,10,2,11,1,29,2,3,0,275,87481190,7.699219,14.703125,24.796875,27.500000,4.199219,1.500000,280.75,2017.0,10.000000,7.500000,9.898438,19.296875,2.000000,2.500000,1.299805,280.500,0.029907,281,0,0,0,0,0,0,10,5,0.795898,0.468506,3.488122e+07,2.458984,14.921875,29.546875,4.449219,0.452637,275.5,0,0,0,0,1,2,6627,6207,0.980957,0.722168,5.549898e+07,4.695312,inf,inf,4.320312,1.297852,inf,inf,inf,4.476562,inf,inf,1.994141,3.142578,1.089844,inf,0.032959,281,0,0,0,0,1,2,11.953125,3.242188,0.759277,0.448486,3.829811e+08
1,8,160417,1507029620497,0,0,0,0,4,2,2017-10-03 11:20:20,3,11,20,20,3,1,276,281,1506942089000,0.025864,2017-10-02 11:01:29,2017,10,2,11,1,29,2,3,0,275,87531497,6.250000,7.625000,22.625000,27.375000,1.875000,0.750000,279.25,2017.0,10.000000,5.375000,13.500000,22.125000,2.000000,3.500000,1.625000,278.500,0.042694,281,0,0,0,0,4,0,8,3,0.834473,0.489746,5.448148e+07,2.458984,14.921875,29.546875,4.449219,0.452637,275.5,0,0,0,0,1,2,6627,6207,0.980957,0.722168,5.549898e+07,4.695312,inf,inf,4.320312,1.297852,inf,inf,inf,4.476562,inf,inf,1.994141,3.142578,1.089844,inf,0.032959,281,0,0,0,0,1,2,11.953125,3.242188,0.759277,0.448486,3.829811e+08
2,10,160417,1507029634489,0,0,0,0,6,0,2017-10-03 11:20:34,3,11,20,34,3,1,276,281,1506942089000,0.025864,2017-10-02 11:01:29,2017,10,2,11,1,29,2,3,0,275,87545489,6.191406,13.359375,25.468750,31.875000,3.660156,1.063477,279.25,2017.0,9.937500,7.253906,10.976562,24.343750,2.000000,2.914062,1.552734,278.250,0.030975,231,0,0,0,0,6,0,47,5,0.827148,0.509766,8.218278e+07,2.458984,14.921875,29.546875,4.449219,0.452637,275.5,0,0,0,0,1,2,6627,6207,0.980957,0.722168,5.549898e+07,4.695312,inf,inf,4.320312,1.297852,inf,inf,inf,4.476562,inf,inf,1.994141,3.142578,1.089844,inf,0.032959,281,0,0,0,0,1,2,11.953125,3.242188,0.759277,0.448486,3.829811e+08
3,10,160417,1507047105515,0,0,0,0,6,0,2017-10-03 16:11:45,3,16,11,45,4,1,276,281,1506942089000,0.025864,2017-10-02 11:01:29,2017,10,2,11,1,29,2,3,0,275,105016515,6.191406,13.359375,25.468750,31.875000,3.660156,1.063477,279.25,2017.0,9.937500,7.253906,10.976562,24.343750,2.000000,2.914062,1.552734,278.250,0.030975,231,0,0,0,0,6,0,47,5,0.827148,0.5097

## 6.2 特征统计信息

In [ ]:
hist_click_df.describe()

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,day,hour,minute,second,time_bucket,weekday,yearday,category_id,prod_at_ts_ms,price,prod_year,prod_month,prod_day,prod_hour,prod_minute,prod_second,prod_season,prod_time_bucket,prod_weekday,prod_yearday,freshness,u_mean_day,u_mean_hour,u_mean_minute,u_mean_second,u_mean_time_bucket,u_mean_weekday,u_mean_yearday,u_mean_prod_year,u_mean_prod_month,u_mean_prod_day,u_mean_prod_hour,u_mean_prod_minute,u_mean_prod_season,u_mean_prod_time_bucket,u_mean_prod_weekday,u_mean_prod_yearday,u_mean_price,u_mode_category_id,u_mode_environment,u_mode_device_group,u_mode_os,u_mode_province,u_mode_city,u_mode_referrer_type,u_active_nums,u_active_day_nums,u_mean_rev_ua_time,u_ua_score,u_mean_freshness,i_mean_day,i_mean_hour,i_mean_minute,i_mean_time_bucket,i_mean_weekday,i_mean_yearday,i_mode_environment,i_mode_device_group,i_mode_os,i_mode_province,i_mode_city,i_mode_referrer_type,i_active_nums,i_active_day_nums,i_mean_rev_ua_time,i_ua_score,i_mean_freshness,c_mean_u_mean_day,c_mean_u_mean_hour,c_mean_u_mean_minute,c_mean_u_mean_time_bucket,c_mean_u_mean_weekday,c_mean_u_mean_yearday,c_mean_u_mean_prod_year,c_mean_u_mean_prod_month,c_mean_u_mean_prod_day,c_mean_u_mean_prod_hour,c_mean_u_mean_prod_minute,c_mean_u_mean_prod_season,c_mean_u_mean_prod_time_bucket,c_mean_u_mean_prod_weekday,c_mean_u_mean_prod_yearday,c_mean_u_mean_price,c_mode_u_mode_category_id,c_mode_u_mode_environment,c_mode_u_mode_device_group,c_mode_u_mode_os,c_mode_u_mode_province,c_mode_u_mode_city,c_mode_u_mode_referrer_type,c_mean_u_active_nums,c_mean_u_active_day_nums,c_mean_u_mean_rev_ua_time,c_mean_u_ua_score,c_mean_u_mean_freshness
count,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1390153.00,1390153.0,1390153.0,1.390153e+06,1.390153e+06,1.390153e+06,1390153.0,1.390153e+06,1.390153e+06,1390153.00,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1390153.00,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1390153.0,1390153.0,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1390153.0,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06,1.390153e+06
mean,1.243686e+05,1.944529e+05,1.507492e+12,4.838388e-02,4.020313e-01,5.838012e-01,1.558159e-01,6.631805e+00,1.251020e+00,8.215457e+00,1.407668e+01,2.960805e+01,2.949429e+01,4.181814e+00,2.278120e+00,2.812157e+02,3.056381e+02,1.506760e+12,0.000000e+00,2.016983e+03,9.929924e+00,8.055305e+00,1.238473e+01,2.722265e+01,2.827318e+01,1.988032e+00,3.254712e+00,2.236585e+00,2.789279e+02,7.314837e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,3.118177e+02,4.773791e-02,4.005221e-01,5.818669e-01,1.542765e-01,6.653641e+00,1.283455e+00,1.963019e+01,4.395782e+00,NaN,NaN,7.314837e+08,NaN,NaN,NaN,NaN,NaN,NaN,4.718905e-04,1.826461e-01,1.136112e-01,1.262451e-03,2.510828e+00,1.412923e+00,3.266645e+03,2.987535e+03,NaN,NaN,7.314837e+08,inf,inf,inf,NaN,NaN,inf,inf,inf,inf,inf,inf,NaN,NaN,NaN,inf,0.000000e+00,3.138070e+02,3.258634e-04,1.741226e-01,1.127696e-01,1.340860e-03,2.529747e+00,1.447609e+00,NaN,NaN,NaN,NaN,7.314837e+08
std,8.367476e

## 6.3 填充信息统计 

In [ ]:
log_table.isna().apply(sum, axis=0)

user_id             0
item_id             0
timestamp_ms        0
environment         0
device_group        0
os                  0
province            0
city                0
referrer_type       0
timestamp_s         0
day                 0
hour                0
minute              0
second              0
time_bucket         0
weekday             0
yearday             0
category_id         0
prod_at_ts_ms       0
price               0
prod_at_ts_s        0
prod_year           0
prod_month          0
prod_day            0
prod_hour           0
prod_minute         0
prod_second         0
prod_season         0
prod_time_bucket    0
prod_weekday        0
prod_yearday        0
freshness           0
dtype: int64

## 6.4 u侧特征

In [ ]:
user_info

,user_id,u_mean_day,u_mean_hour,u_mean_minute,u_mean_second,u_mean_time_bucket,u_mean_weekday,u_mean_yearday,u_mean_prod_year,u_mean_prod_month,u_mean_prod_day,u_mean_prod_hour,u_mean_prod_minute,u_mean_prod_season,u_mean_prod_time_bucket,u_mean_prod_weekday,u_mean_prod_yearday,u_mean_price,u_mode_category_id,u_mode_environment,u_mode_device_group,u_mode_os,u_mode_province,u_mode_city,u_mode_referrer_type,u_active_nums,u_active_day_nums,u_mean_rev_ua_time,u_ua_score,u_mean_freshness
0,0,7.699219,14.703125,24.796875,27.500000,4.199219,1.500000,280.75,2017.0,10.000000,7.500000,9.898438,19.296875,2.0,2.500000,1.299805,280.50,0.029907,281,0,0,0,0,0,0,10,5,0.795898,0.468506,34881216.0
1,1,8.382812,11.898438,31.203125,30.828125,3.000000,1.538086,281.50,2017.0,9.898438,9.945312,13.664062,28.359375,2.0,3.718750,2.179688,279.75,0.028351,375,0,0,0,0,1,1,39,7,0.730469,0.448730,124511744.0
2,2,3.000000,11.000000,21.000000,23.000000,3.000000,1.000000,276.00,2017.0,10.000000,3.000000,6.000000,59.000000,2.0,1.000000,1.000000,276.00,0.023315,354,0,0,0,0,2,0,1,1,0.000000,0.000000,15720061.0
3,3,6.500000,13.000000,23.421875,28.250000,3.583984,2.750000,279.50,2017.0,10.000000,6.082031,12.335938,24.078125,2.0,3.083984,2.333984,279.00,0.029160,250,0,0,0,0,1,2,12,4,0.839355,0.495361,38362308.0
4,4,5.167969,12.164062,35.656250,32.500000,3.166016,0.833496,278.25,2017.0,10.000000,4.332031,12.335938,11.500000,2.0,3.333984,1.166992,277.25,0.029404,412,0,0,0,0,1,0,6,2,0.905273,0.529785,72853456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,7.363281,12.421875,38.093750,29.812500,3.636719,1.757812,280.25,2017.0,10.000000,6.667969,10.968750,26.906250,2.0,2.666016,1.272461,279.75,0.029251,375,0,0,0,0,0,0,33,6,0.781250,0.474609,66130468.0
249996,249996,8.500000,14.500000,38.000000,36.500000,4.000000,3.000000,281.50,2017.0,10.000000,8.164062,11.664062,23.828125,2.0,3.000000,2.666016,281.25,0.032745,281,0,0,0,0,0,0,6,2,0.708496,0.415527,39861668.0
249997,249997,3.833984,12.835938,24.828125,41.843750,3.333984,1.833008,276.75,2017.0,10.000000,3.333984,11.500000,24.328125,2.0,2.833984,1.333008,276.25,0.031189,428,0,0,0,0,3,0,6,3,0.865723,0.506836,48045888.0
249998,249998,3.000000,11.000000,18.000000,56.000000,3.000000,1.000000,276.00,2017.0,10.000000,2.000000,20.000000,8.000000,2.0,6.000000,0.000000,275.00,0.026306,299,0,1,1,0,16,3,1,1,0.000000,0.000000,54608440.0


In [ ]:
user_info.to_csv(os.path.join(save_dir, 'user_info.csv'), index=False)

## 6.5 i侧特征

In [ ]:
item_info

,item_id,category_id,prod_at_ts_ms,price,prod_at_ts_s,prod_year,prod_month,prod_day,prod_hour,prod_minute,prod_second,prod_season,prod_time_bucket,prod_weekday,prod_yearday,i_mean_day,i_mean_hour,i_mean_minute,i_mean_time_bucket,i_mean_weekday,i_mean_yearday,i_mode_environment,i_mode_device_group,i_mode_os,i_mode_province,i_mode_city,i_mode_referrer_type,i_active_nums,i_active_day_nums,i_mean_rev_ua_time,i_ua_score,i_mean_freshness
0,0,0,1513144419000,0.025116,2017-12-13 05:53:39,2017,12,13,5,53,39,3,1,2,347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1405341936000,0.028244,2014-07-14 12:45:36,2014,7,14,12,45,36,1,3,0,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1408667706000,0.037384,2014-08-22 00:35:06,2014,8,22,0,35,6,1,7,4,234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1408468313000,0.034393,2014-08-19 17:11:53,2014,8,19,17,11,53,1,5,1,231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1407071171000,0.024216,2014-08-03 13:06:11,2014,8,3,13,6,11,1,4,6,215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364042,364042,460,1434034118000,0.021530,2015-06-11 14:48:38,2015,6,11,14,48,38,1,4,3,162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364043,364043,460,1434148472000,0.069214,2015-06-12 22:34:32,2015,6,12,22,34,32,1,6,4,163,10.5,16.75,24.25,4.5,1.5,283.5,0.0,0.0,0.0,0.0,10.0,1.0,4.0,4.0,0.557129,0.282959,7.355010e+10
364044,364044,460,1457974279000,0.026459,2016-03-14 16:51:19,2016,3,14,16,51,19,0,4,0,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364045,364045,460,1515964737000,0.018829,2018-01-14 21:18:57,2018,1,14,21,18,57,3,6,6,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_timestamp(time_str):
  timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
  timeStamp = int(time.mktime(timeArray))
  return timeStamp

In [ ]:
item_info['prod_at_ts_s'] = item_info['prod_at_ts_s'].map(get_timestamp)

In [ ]:
item_info['i_mode_referrer_type'].isna().sum()

333411

In [ ]:
item_info.to_csv(os.path.join(save_dir, 'item_info.csv'), index=False)

## 6.6 consumer侧特征

In [ ]:
consumer_info

,item_id,c_mean_u_mean_day,c_mean_u_mean_hour,c_mean_u_mean_minute,c_mean_u_mean_time_bucket,c_mean_u_mean_weekday,c_mean_u_mean_yearday,c_mean_u_mean_prod_year,c_mean_u_mean_prod_month,c_mean_u_mean_prod_day,c_mean_u_mean_prod_hour,c_mean_u_mean_prod_minute,c_mean_u_mean_prod_season,c_mean_u_mean_prod_time_bucket,c_mean_u_mean_prod_weekday,c_mean_u_mean_prod_yearday,c_mean_u_mean_price,c_mode_u_mode_category_id,c_mode_u_mode_environment,c_mode_u_mode_device_group,c_mode_u_mode_os,c_mode_u_mode_province,c_mode_u_mode_city,c_mode_u_mode_referrer_type,c_mean_u_active_nums,c_mean_u_active_day_nums,c_mean_u_mean_rev_ua_time,c_mean_u_ua_score,c_mean_u_mean_freshness
0,160417,4.695312,inf,inf,4.320312,1.297852,inf,inf,inf,4.476562,inf,inf,1.994141,3.142578,1.089844,inf,0.032959,281,0,0,0,0,1,2,11.953125,3.242188,0.759277,0.448486,3.829811e+08
1,5408,5.425781,11.531250,28.812500,3.130859,1.716797,inf,inf,9.953125,5.292969,9.140625,26.562500,1.995117,2.339844,1.522461,inf,0.027603,281,0,0,0,0,1,0,13.109375,3.517578,0.735352,0.435303,3.986711e+08
2,50823,5.765625,13.671875,30.359375,3.890625,1.690430,inf,inf,9.937500,5.839844,10.039062,30.515625,1.992188,2.445312,1.628906,inf,0.031647,281,0,0,0,0,1,2,15.656250,4.023438,0.769531,0.456543,5.036617e+08
3,123909,inf,inf,inf,4.382812,2.585938,inf,inf,inf,inf,inf,inf,1.993164,2.904297,2.458984,inf,0.031647,250,0,0,0,0,1,2,14.804688,3.892578,0.709961,0.421631,4.451103e+08
4,168623,inf,inf,inf,4.195312,2.486328,inf,inf,inf,inf,inf,inf,1.994141,3.851562,2.199219,inf,0.029266,281,0,0,0,0,1,2,14.156250,3.955078,0.732422,0.434326,4.122668e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30631,353518,6.601562,18.593750,23.296875,5.699219,1.099609,279.50,2017.0,9.296875,10.500000,13.296875,28.203125,1.700195,3.500000,1.099609,262.000,0.030685,281,0,0,0,0,1,2,10.000000,3.000000,0.767578,0.452148,1.530785e+09
30632,44769,4.265625,12.335938,34.468750,3.400391,1.799805,277.25,2017.0,8.531250,7.800781,14.203125,28.328125,1.799805,3.800781,0.799805,236.125,0.029083,455,0,0,0,0,19,0,15.000000,3.000000,0.927246,0.548340,3.547570e+09
30633,42354,4.265625,12.335938,34.468750,3.400391,1.799805,277.25,2017.0,8.531250,7.800781,14.203125,28.328125,1.799805,3.800781,0.799805,236.125,0.029083,455,0,0,0,0,19,0,15.000000,3.000000,0.927246,0.548340,3.547570e+09
30634,287083,9.117188,13.296875,31.937500,3.587891,2.175781,282.00,2017.0,9.882812,8.765625,11.703125,28.640625,2.000000,2.941406,2.175781,278.250,0.029251,412,0,0,0,0,1,2,17.000000,7.000000,0.704590,0.419922,5.911603e+09


In [ ]:
consumer_info.to_csv(os.path.join(save_dir, 'consumer_info.csv'), index=False)

## 6.7 ctx info

In [ ]:
ctx_fea_list = ['user_id', 'item_id', 'timestamp_ms', 'environment', 
         'device_group', 'os', 'province',	'city', 'referrer_type', 'timestamp_s',
         'day', 'hour', 'minute', 'second', 'time_bucket', 'weekday', 'yearday', 'freshness']
ctx_info = log_table[ctx_fea_list]
hist_click_df = hist_click_df[ctx_fea_list]
last_click_df = last_click_df[ctx_fea_list]

In [ ]:
ctx_info

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s,day,hour,minute,second,time_bucket,weekday,yearday,freshness
0,0,160417,1507029570190,0,0,0,0,0,0,2017-10-03 11:19:30,3,11,19,30,3,1,276,87481190
1,0,5408,1507029571478,0,0,0,0,0,0,2017-10-03 11:19:31,3,11,19,31,3,1,276,35314478
2,0,50823,1507029601478,0,0,0,0,0,0,2017-10-03 11:20:01,3,11,20,1,3,1,276,15987478
3,1,157770,1507029532200,0,0,0,0,1,1,2017-10-03 11:18:52,3,11,18,52,3,1,276,45597200
4,1,96613,1507029671831,0,0,0,0,1,1,2017-10-03 11:21:11,3,11,21,11,3,1,276,91227831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630628,228075,70758,1508211323220,0,1,1,0,1,2,2017-10-17 03:35:23,17,3,35,23,0,1,290,23159220
1630629,242176,331116,1508211542618,0,1,1,0,1,0,2017-10-17 03:39:02,17,3,39,2,0,1,290,133923618
1630630,242176,234481,1508211850103,0,1,1,0,1,0,2017-10-17 03:44:10,17,3,44,10,0,1,290,30278103
1630631,242176,211442,1508212189949,0,1,1,0,1,0,2017-10-17 03:49:49,17,3,49,49,0,1,290,24469949


In [ ]:
ctx_info['timestamp_s'] = ctx_info['timestamp_s'].map(get_timestamp)
hist_click_df['timestamp_s'] = hist_click_df['timestamp_s'].map(get_timestamp)
last_click_df['timestamp_s'] = last_click_df['timestamp_s'].map(get_timestamp)

<ipython-input-109-785e27afa191>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctx_info['timestamp_s'] = ctx_info['timestamp_s'].map(get_timestamp)
<ipython-input-109-785e27afa191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_click_df['timestamp_s'] = hist_click_df['timestamp_s'].map(get_timestamp)


In [ ]:
ctx_info.to_csv(os.path.join(save_dir, 'ctx_info.csv'), index=False)

In [ ]:
hist_click_df.to_csv(os.path.join(save_dir, 'hist_click_df.csv'), index=False)
last_click_df.to_csv(os.path.join(save_dir, 'last_click_df.csv'), index=False)